In [105]:
!pip install numpy pandas seaborn


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


**Задача**: Выбор VR-игры для разработки в рамках дисциплины <<Разработка приложений виртуальной реальности>> (7-й семестр).

**Альтернативы**:
   - Космический квест для обучения троичной логике
   - Визуальная новелла - симулятор студента
   - VR-хоррор про выживание в Backrooms
   - Психологический хоррор про применение ИИ
   - Виртуальная галерея объектов искусства

**Критерии**:
- Простота реализации
- Актуальность
- Оригинальность
- Образовательная ценность
- VR-потенциал

In [106]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

alternative_names = [
    "Space Quest",
    "Student Simulator",
    "Backrooms Horror",
    "Psychological Horror",
    "Art Gallery"
]

criteria_names = [
    "Simplicity",
    "Relevance",
    "Originality",
    "Education",
    "VR Potential"
]

Построение матрицы попарных сравнений:

- C₁ vs C₂ (Простота vs Актуальность): Простота немного важнее** → **3**
- C₁ vs C₃ (Простота vs Оригинальность): **Простота умеренно важнее** → **3**
- C₁ vs C₄ (Простота vs Образовательная ценность): **Образовательная ценность немного важнее** → **1/3**
- C₁ vs C₅ (Простота vs VR-потенциал): **VR-потенциал умеренно важнее** → **1/3**
- C₂ vs C₃ (Актуальность vs Оригинальность): **Оригинальность немного важнее** → **1/2**
- C₂ vs C₄ (Актуальность vs Образовательная ценность): **Образовательная ценность сильно важнее** → **1/5**
- C₂ vs C₅ (Актуальность vs VR-потенциал): **VR немного важнее** → **1/2**
- C₃ vs C₄ (Оригинальность vs Образовательная ценность): **Образование умеренно важнее** → **1/3**
- C₃ vs C₅ (Оригинальность vs VR-потенциал): **VR немного важнее** → **1/2**
- C₄ vs C₅ (Образование vs VR-потенциал): **Образование умеренно важнее** → **3**


In [107]:
def build_pairwise_comparison_matrix(topright: np.ndarray) -> np.ndarray:
    if topright.ndim != 2 or topright.shape[0] != topright.shape[1]:
        raise ValueError("The provided topright matrix should be a square matrix.")
    
    dim = topright.shape[0]
    
    topright_half = topright * np.tri(dim).T
    topright_half = topright_half.astype(dtype=np.float64)
        
    bottomleft_half = np.zeros_like(topright_half)
    mask = topright_half != 0
    bottomleft_half[mask] = 1 / topright_half[mask]
    bottomleft_half = bottomleft_half.T
    
    result = np.eye(dim) + topright_half + bottomleft_half
    return result

crit_comp_matrix = build_pairwise_comparison_matrix(np.array([
    [0, 3, 3,   1/3, 1/3],
    [0, 0, 1/2, 1/5, 1/2],
    [0, 0, 0,   1/3, 1/2],
    [0, 0, 0,   0,   3  ],
    [0, 0, 0,   0,   0  ],
]))
crit_comp_matrix

array([[1.        , 3.        , 3.        , 0.33333333, 0.33333333],
       [0.33333333, 1.        , 0.5       , 0.2       , 0.5       ],
       [0.33333333, 2.        , 1.        , 0.33333333, 0.5       ],
       [3.        , 5.        , 3.        , 1.        , 3.        ],
       [3.        , 2.        , 2.        , 0.33333333, 1.        ]])

Проверим относительную согласованность матрицы:

In [108]:
def ahp_random_index(n: int = 5) -> float:
    ri_table = [
        0, 0, 0, 0.58, 0.9, 1.12, 1.24, 1.32, 1.41, 1.45, 1.49
    ]
    try:
        return ri_table[n]
    except IndexError:
        raise ValueError(f"AHP Random Index currently unavailable for matrix size {n}.") 

ri = ahp_random_index(5)
ri

1.12

In [109]:
def consistency_ratio(comparison_matrix: np.ndarray) -> float:
    if (comparison_matrix.ndim != 2 or
            comparison_matrix.shape[0] != comparison_matrix.shape[1]):
        raise ValueError("The provided matrix should be a square matrix.")
    
    dim = comparison_matrix.shape[0]
    
    eigvals = np.linalg.eig(comparison_matrix).eigenvalues
    l_max = np.max(np.real(eigvals))
    
    ci = (l_max - dim) / (dim - 1)
    ri = ahp_random_index(dim)
    return ci / ri if ri > 0 else 0
    
cr = consistency_ratio(crit_comp_matrix)
if cr > 0.1:
    raise UserWarning(f"Consistency ratio must not be greater than 0.1 (currently {cr}).")
cr

np.float64(0.08236304922521218)

Оценка вектора весов критериев:

In [110]:
def ahp_weights_eigen(matrix: np.ndarray) -> np.ndarray:
    eigvals, eigvecs = np.linalg.eig(matrix)
    max_index = np.argmax(np.real(eigvals))
    principal_vector = np.real(eigvecs[:, max_index])
    normalized_weights = principal_vector / principal_vector.sum()
    return normalized_weights

ahp_weights = ahp_weights_eigen(crit_comp_matrix)
ahp_weights_eigen(crit_comp_matrix)

array([0.16992735, 0.07060938, 0.10403539, 0.42627555, 0.22915233])

In [111]:
def ahp_weights_row_norm(matrix: np.ndarray) -> np.ndarray:
    norm_matrix = matrix / matrix.sum(axis=0)
    weights = norm_matrix.mean(axis=1)
    return weights

ahp_weights_row_norm(crit_comp_matrix)

array([0.17820173, 0.0715384 , 0.10957054, 0.42175093, 0.21893839])

Сравнительная оценка альтернатив по каждому критерию

In [147]:
criteriawise_alternative_comparison_matrices: dict[str, np.ndarray] = {
    'Simplicity': 
        build_pairwise_comparison_matrix(np.array([
            [0, 1/2, 2  , 3  , 1/5],
            [0, 0  , 3  , 5  , 1/2],
            [0, 0  , 0  , 2  , 1/5],
            [0, 0  , 0  , 0  , 1/6],
            [0, 0  , 0  , 0  , 0  ],
        ])),
    'Relevance':
        build_pairwise_comparison_matrix(np.array([
            [0, 3  , 2  , 1/2, 5  ],
            [0, 0  , 1/2, 1/5, 2  ],
            [0, 0  , 0  , 1/3, 3  ],
            [0, 0  , 0  , 0  , 7  ],
            [0, 0  , 0  , 0  , 0  ],
        ])),
    'Originality':
        build_pairwise_comparison_matrix(np.array([
            [0, 5  , 7  , 3  , 9  ],
            [0, 0  , 3  , 1/3, 5  ],
            [0, 0  , 0  , 1/5, 3  ],
            [0, 0  , 0  , 0  , 7  ],
            [0, 0  , 0  , 0  , 0  ],
        ])),
    'Education':
        build_pairwise_comparison_matrix(np.array([
            [0, 5  , 7  , 3  , 1/3],
            [0, 0  , 3  , 1/3, 1/7],
            [0, 0  , 0  , 1/5, 1/8],
            [0, 0  , 0  , 0  , 1/5],
            [0, 0  , 0  , 0  , 0  ],
        ])),
    'VR Potential':
        build_pairwise_comparison_matrix(np.array([
            [0, 7  , 3  , 9  , 5  ],
            [0, 0  , 1/5, 3  , 1/3],
            [0, 0  , 0  , 7  , 3  ],
            [0, 0  , 0  , 0  , 1/5],
            [0, 0  , 0  , 0  , 0  ],
        ])),
}

for cm_name, cm in criteriawise_alternative_comparison_matrices.items():
    cr = consistency_ratio(cm)
    print(f"{cm_name}: {consistency_ratio(cm)}")
    assert cr < 0.1, f'Consistency ratio for criterion {cm_name}: {cr} >= 0.1'

Simplicity: 0.019546058035832932
Relevance: 0.006254980971022648
Originality: 0.05300786863556045
Education: 0.05946284820946398
VR Potential: 0.05300786863555986


In [148]:
local_weights = []

for crit in criteria_names:
    comp_matrix = criteriawise_alternative_comparison_matrices[crit]
    local_weight_vector = ahp_weights_eigen(comp_matrix)
    local_weights.append(local_weight_vector)
    
overall_alternative_to_criteria_matrix = np.array(local_weights)
overall_alternative_to_criteria_matrix

array([[0.13529807, 0.25246326, 0.08630608, 0.0532177 , 0.47271489],
       [0.26192318, 0.08867844, 0.1523594 , 0.44464814, 0.05239084],
       [0.51281281, 0.12897642, 0.06337653, 0.26149906, 0.03333518],
       [0.26325206, 0.06397846, 0.03479308, 0.13013783, 0.50783857],
       [0.51281281, 0.06337653, 0.26149906, 0.03333518, 0.12897642]])

In [154]:
# Multiply each local row by its criterion weight
global_scores = (ahp_weights[:, None] * overall_alternative_to_criteria_matrix).sum(axis=0)
global_scores

array([0.32456593, 0.10437539, 0.10677173, 0.13075804, 0.33352892])

In [155]:
# Display result
for alt, score in sorted(
        zip(alternative_names, global_scores),
        key=lambda k: k[1],
        reverse=True
):
    print(f"{alt:<20}: {score:.4f}")
    
assert np.abs(global_scores.sum() - 1) < 1e-6

Art Gallery         : 0.3335
Space Quest         : 0.3246
Psychological Horror: 0.1308
Backrooms Horror    : 0.1068
Student Simulator   : 0.1044
